In [ ]:
# @title Check environment
!env | sort
!python --version
!pip list
!nvidia-smi
!nvcc --version

In [ ]:
# @title Clone
!git clone https://github.com/Jaid/kohya-gradio.git
%cd kohya-gradio

In [ ]:
!python -m pip --disable-pip-version-check install --upgrade pip
%pip install --disable-pip-version-check --requirement requirements_colab.txt --extra-index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://pypi.nvidia.com


In [ ]:
%mkdir --parents private
import requests
def download(url, target):
  print(f"{url} → {target}")
  response = requests.get(url, allow_redirects=True)
  response.raise_for_status()
  with open(target, 'wb') as output_file:
    output_file.write(response.content)
download('https://huggingface.co/stabilityai/sdxl-vae/resolve/main/sdxl_vae.safetensors', 'private/vae.safetensors')

In [ ]:
# @title Check CUDA support in PyTorch
!pip show torch
import torch
print(torch.cuda.is_available())
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
!pip show tensorflow
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

In [ ]:
wandbKey = '' # @param {type:"string"}
!accelerate launch sdxl_train_network.py --pretrained_model_name_or_path stabilityai/stable-diffusion-xl-base-1.0 --vae private/vae.safetensors --train_data_dir img --output_dir out/model --logging_dir out/log --resolution 1024,1024 --save_model_as safetensors --text_encoder_lr 0.0001 --unet_lr 0.0001 --network_dim 64 --network_alpha 32 --output_name trained --max_train_steps 80 --no_half_vae --caption_extension .txt --cache_latents --network_module lycoris.kohya --network_args preset=full algo=full rank_dropout=0 module_dropout=0 use_tucker=False use_scalar=False rank_dropout_scale=True train_norm=True --optimizer_type adamw --save_every_n_epochs 1 --lr_scheduler constant --xformers --mixed_precision bf16 --save_precision bf16 --metadata_title SinansWoche --log_with wandb --wandb_api_key {wandbKey}